In [1]:
!pip install nltk torch transformers evaluate bert_score rouge_score
import sys
import nltk


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
#Load groundtruth and predictions
prediction_list=[]
groundtruth_list=[]

prediction_length_sum=[]
groundtruth_length_sum=[]
dist_bigram_list=[]
dist_trigram_list=[]
all_prediction_text=""

In [3]:
with open("predictions.tsv",'r') as file:
    header=file.readline()

    for line in file:
        parts=line.strip().split('\t')
        prediction=parts[1]
        groundtruth=parts[2]
        
        if(len(nltk.word_tokenize(prediction))==0):
            continue

        prediction_list.append(prediction)
        groundtruth_list.append(groundtruth)

        
        pred_tokens=nltk.word_tokenize(prediction)
        ground_tokens=nltk.word_tokenize(groundtruth)
        
        prediction_length_sum.append(len(pred_tokens))
        groundtruth_length_sum.append(len(ground_tokens))
            
        dist_bigram_list.append(len(list(set(list(nltk.bigrams(pred_tokens)))))/len(pred_tokens))
        dist_trigram_list.append(len(list(set(list(nltk.trigrams(pred_tokens)))))/len(pred_tokens))
        
        all_prediction_text+=" "+prediction

In [4]:
# Evaluation
from evaluate import load
from statistics import mean

perplexity = load("perplexity", module_type="metric")
bleu = load("bleu")
bertscore = load("bertscore")
rouge = load('rouge')

In [5]:
groundtruth_lol=[[x] for x in groundtruth_list]

results = perplexity.compute(predictions=prediction_list, model_id='gpt2')
print("Mean Perplexity with GPT2:",results['mean_perplexity'])
results = bleu.compute(predictions=prediction_list, references=groundtruth_lol,max_order=2)
print("BLEU-2:",results['bleu'])
results = bleu.compute(predictions=prediction_list, references=groundtruth_lol,max_order=3)
print("BLEU-3:",results['bleu'])
results = bleu.compute(predictions=prediction_list, references=groundtruth_lol,max_order=4)
print("BLEU-4:",results['bleu'])
results = bertscore.compute(predictions=prediction_list, references=groundtruth_list, lang="en")
print("BERT Score [Precision,Recall,F1]:",mean(results['precision']),mean(results['recall']),mean(results['f1']))
results = rouge.compute(predictions=prediction_list,references=groundtruth_list)
print("ROUGE",results)

Using pad_token, but it is not set yet.


  0%|          | 0/442 [00:00<?, ?it/s]

Mean Perplexity with GPT2: 250.67663101611436
BLEU-2: 0.3867942092676606
BLEU-3: 0.36073829243562455
BLEU-4: 0.3326578445520932
BERT Score [Precision,Recall,F1]: 0.9215647799867027 0.8893884407725637 0.9048125814053667
ROUGE {'rouge1': 0.4888599938118462, 'rouge2': 0.3725515196790248, 'rougeL': 0.45582259132216413, 'rougeLsum': 0.45591557544161865}


In [6]:
print("Groundtruth avg length:",sum(groundtruth_length_sum)/len(groundtruth_length_sum))
print("Prediction avg length:",sum(prediction_length_sum)/len(prediction_length_sum))

print("Dist-2",sum(dist_bigram_list)/len(dist_bigram_list))
print("Dist-3",sum(dist_trigram_list)/len(dist_trigram_list))

all_prediction_tokens=nltk.word_tokenize(all_prediction_text)
print("Dist-2 corpus",len(list(set(list(nltk.bigrams(all_prediction_tokens)))))/len(all_prediction_tokens))
print("Dist-3 corpus",len(list(set(list(nltk.trigrams(all_prediction_tokens)))))/len(all_prediction_tokens))

Groundtruth avg length: 14.396910868641065
Prediction avg length: 9.40357092248831
Dist-2 0.8392215018686178
Dist-3 0.7108386936695468
Dist-2 corpus 0.12053766519491871
Dist-3 corpus 0.24071367218697728
